In [1]:
import pandas as pd
import re

In [ ]:
# 1. 读取CSV文件 (请替换为你的文件名)
# 假设你的文件名为 job_data.csv，编码可能是 utf-8 或 shift-jis
try:
    df = pd.read_csv('jd_data_raw.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('jd_data_raw.csv', encoding='shift-jis')

# 定义清洗函数
def process_job_text(text):
    if not isinstance(text, str):
        return pd.Series(['', '', '', '', '', '', '', '', '', '', ''])
    
    # === 1. 数据清洗 (Noise Removal) ===
    # 移除开头的 "NEW " (包含可能的引号和空格)
    clean_text = re.sub(r'^"?NEW\s+', '', text)
    
    # 移除 "応募する あとからしたい方は 気になる" 及其之后的内容
    # 使用正则表达式匹配，允许中间有不同数量的空格
    split_pattern = r'(?:最短\s*[０-９0-9]+\s*分で完了！\s*)?応募する\s+あとからしたい方は\s+気になる'
    match = re.search(split_pattern, clean_text)
    if match:
        clean_text = clean_text[:match.start()]
    
    # 如果没有匹配到长句，尝试匹配简短的结尾标记，防止漏网
    else:
        # 有时候结尾可能略有不同，这里做一个兜底
        alt_split = clean_text.find("応募する あとからしたい方は")
        if alt_split != -1:
            clean_text = clean_text[:alt_split]

    # === 2. 提取职位标题和公司名 ===
    # 逻辑：找到第一组常见的“招聘标签”，标签前面的部分就是【职位+公司】
    # 常见标签：業界未経験OK, 年間休日, 土曜出勤, 正社員, 想定年収, 勤務地, 残業手当
    tags_pattern = r'(?:業界未経験|年間休日|土曜出勤|日曜出勤|残業手当|正社員|想定年収|休日\s*年間|勤務地)'
    
    job_title = ""
    company_name = ""
    
    # 查找标签前的内容
    header_match = re.search(r'^(.*?)[\s　]+(?=' + tags_pattern + ')', clean_text)
    
    if header_match:
        header_text = header_match.group(1).strip()
        # 将头部文本按空格（全角或半角）分割
        parts = re.split(r'[\s　]+', header_text)
        
        if len(parts) >= 2:
            # 假设最后一个部分是公司名（通常公司名中间没有空格，而职位描述很长且有空格）
            company_name = parts[-1]
            # 前面的部分合并为职位名
            job_title = " ".join(parts[:-1])
        else:
            # 如果分割不出来，说明可能只有一个长字符串
            job_title = parts[0]
            company_name = "提取失败" # 需人工检查
    else:
        # 如果找不到标签，尝试直接按前两个大块分割（兜底策略）
        parts = re.split(r'[\s　]+', clean_text[:100]) # 只看前100字
        if len(parts) > 1:
            job_title = parts[0]
            company_name = parts[1]

    # === 3. 提取在宅/Remote 关键词 ===
    # 截取关键词前后10个字符，方便判断语境
    remote_keywords = re.findall(r'(.{0,10}(?:在宅|リモート|テレワーク).{0,10})', clean_text)
    remote_info = " || ".join(remote_keywords)

    # === 4. 提取特定段落 ===
    def extract_between(source, start_str, end_str):
        start_idx = source.find(start_str)
        if start_idx == -1:
            return ""
        # 移动索引到关键词之后
        start_idx += len(start_str)
        
        # 从 start_idx 开始往后找 end_str
        end_idx = source.find(end_str, start_idx)
        if end_idx == -1:
            # 如果没找到结束词，可能是在文本末尾，或者结束词被截断了
            # 这里选择截取到文本末尾或者截取一定长度，视情况而定
            # 根据你的数据，通常后面紧接着其他字段，如果找不到end_str，可能需要特殊处理
            # 这里假设如果找不到结束词，就取剩下全部（或留空）
            return source[start_idx:].strip()[:500] # 限制长度防止错误
            
        return source[start_idx:end_idx].strip()

    job_content = extract_between(clean_text, "仕事の内容", "配属先情報")
    dept_info = extract_between(clean_text, "配属先情報", "必要な能力・経験")
    requirements = extract_between(clean_text, "必要な能力・経験", "勤務地") # 注意：勤務地在文中出现多次，find会找最近的一个，符合逻辑
    salary = extract_between(clean_text, "給与", "就業時間")
    work_hours = extract_between(clean_text, "就業時間", "通勤手当")
    selection = extract_between(clean_text, "選考内容", "企業情報")
    
    # 提取员工人数 (在 "従業員数" 后面找数字)
    emp_count_match = re.search(r'従業員数[\s　]*([0-9,]+)', clean_text)
    emp_count = emp_count_match.group(1) if emp_count_match else ""

    return pd.Series([
        job_title, 
        company_name, 
        remote_info, 
        job_content, 
        dept_info, 
        requirements, 
        salary, 
        work_hours, 
        selection, 
        emp_count,
        clean_text # 清洗后的全文（去除了无关尾部）
    ])

# 应用函数到DataFrame
# 假设原文列名是 '内容全文'
columns = [
    '提取_职位名', '提取_公司名', '提取_在宅情况', 
    '提取_工作内容', '提取_配属部门', '提取_必要经验', 
    '提取_薪资', '提取_就业时间', '提取_选考内容', '提取_员工人数',
    '清洗后_内容全文'
]

df[columns] = df['内容全文'].apply(process_job_text)



FileNotFoundError: [Errno 2] No such file or directory: 'job_data_raw.csv'

In [ ]:
display(df.head(2))

,序号,求人票ID,更新时间,职位名称,公司名,员工人数,在宅信息,工作内容,配属先情报,能力经验要求,选考流程
0,1394,j107579424,2026年01月09日,【データ活用/教育枠】佐川急便が抱えるビジネス課題をデータ分析の力で解決,ＳＧシステム株式会社,"2,634名",【魅力】リモートワーク基本 ★入社者のスムーズな定着を目,佐川急便が抱えるビジネス課題を、データ分析の力で解決に導くプロジェクトをお任せします！SQL...,グループDX推進1部 IT業務改革課 ※全体16名 ※現在30テーマ程が走っており、1テーマ...,【必須】・SQLを用いたデータ抽出・加工の実務経験 ・Pythonを用いたデータ分析・機械学...,
1,1395,t107575547,2026年01月09日,【4/1入社限定】データアナリスト/基盤構築エンジニア/自由な働き方可能◎,ＡＭＢＬ株式会社,700名,煙（屋外喫煙可能場所あり） 勤務地備考 在宅勤務可※在宅勤務/リモートワークの利用に,多様な業界で、データ分析基盤の構築から顧客分析・需要予測・価格最適化・市場/売上分析・KPI...,イノベーション事業本部 データサイエンス事業部,【必須】■IT業界での就業経験 （目安：2年以上） 【いずれか必須(1年以上)】■システム開...,


In [ ]:
# 保存
df.to_csv('jd_data_structured.csv', index=False, encoding='utf-8-sig')